# Import required modules and libraries

In [3]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, Activation,Dropout
from tensorflow.keras.utils import to_categorical
import PIL
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load a pre-trained model

In [4]:
pre_trained_model = VGG19(input_shape = (150,150,3),
                          include_top = False,
                          weights='imagenet',
                          classes = 10)

# Freeze the convolutional layers

In [5]:
for layer in pre_trained_model.layers:
    layer.trainable = False

# Build new model based on the pre-trained model

In [6]:
#Adding the final layers to the above base models where the actual classification is done in the dense layers
model = Sequential()
model.add(pre_trained_model) 
model.add(Flatten()) 
model.add(Dense(1024,activation=('relu'),input_dim=512))
model.add(Dense(512,activation=('relu'))) 
model.add(Dense(256,activation=('relu'))) 
model.add(Dropout(.3))
model.add(Dense(128,activation=('relu')))
model.add(Dropout(.2))
model.add(Dense(10,activation=('softmax')))


# from tensorflow.keras.optimizers import RMSprop
# from tensorflow.keras import layers, Model

# x = layers.Flatten()(pre_trained_model.output)

# x = layers.Dense(1024, activation ='relu')(x)
# x = layers.Dropout(0.2)(x)
# x = layers.Dense(10,activation='sigmoid')(x)

# model = Model(pre_trained_model.input, x)

# model.compile(optimizer = RMSprop(lr=0.0001),
#               loss = 'binary_crossentropy',
#               metrics=['accuracy'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 4, 4, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              8389632   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               3

In [8]:
#Defining the hyperparameters
# learn_rate=.001
# sgd=SGD(lr=learn_rate,momentum=.9,nesterov=False)

#Compiling the VGG19 model
model.compile(optimizer=RMSprop(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Prepare dataset for processing

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range = 0.2,
    zoom_range=0.2,
    horizontal_flip =True)

train_generator = train_datagen.flow_from_directory(
    "D:/Dataset/train",
    target_size=(150,150),
    batch_size = 32,
    class_mode='categorical')

Found 55745 images belonging to 10 classes.


In [10]:
val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
    "D:/Dataset/validate",
    target_size=(150, 150),
    batch_size =32,
    class_mode='categorical')

Found 18459 images belonging to 10 classes.


# Train the new model

In [11]:
model.fit(train_generator,
          steps_per_epoch=100,
          epochs=20,
          validation_data=val_generator,
          validation_steps=18459/32)

Epoch 1/20
100/100 [==============================] - 1784s 18s/step - loss: 2.3441 - accuracy: 0.4494 - val_loss: 0.5538 - val_accuracy: 0.8207
Epoch 2/20
100/100 [==============================] - 1569s 16s/step - loss: 0.4493 - accuracy: 0.8655 - val_loss: 0.3095 - val_accuracy: 0.9127
Epoch 3/20
100/100 [==============================] - 1546s 16s/step - loss: 0.4005 - accuracy: 0.8895 - val_loss: 0.2443 - val_accuracy: 0.9347
Epoch 4/20
100/100 [==============================] - 1563s 16s/step - loss: 0.3071 - accuracy: 0.9162 - val_loss: 0.3365 - val_accuracy: 0.9114
Epoch 5/20
100/100 [==============================] - 1550s 16s/step - loss: 0.2679 - accuracy: 0.9302 - val_loss: 0.3052 - val_accuracy: 0.9268
Epoch 6/20
100/100 [==============================] - 1554s 16s/step - loss: 0.3407 - accuracy: 0.9169 - val_loss: 0.3320 - val_accuracy: 0.9231
Epoch 7/20
100/100 [==============================] - 1555s 16s/step - loss: 0.2785 - accuracy: 0.9316 - val_loss: 0.2405 - val_ac

# Save Model

In [14]:
model.save("./VGG19_tf", save_format = 'tf')

INFO:tensorflow:Assets written to: ./VGG19_tf\assets
